In [1]:
import pandas
import scipy

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
infilename = "drivethrurpg-chaosium.csv"
df = pandas.read_csv(infilename).dropna(how='any')
df = df.loc[df['product'] != "ZIP"]
df.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,pdf,color,product,Greg Stafford,number of authors,number of artists
0,RuneQuest Starter Set,https://www.drivethrurpg.com/en/product/370454...,180.0,11/10/21,"RuneQuest, BRP (Basic Roleplaying)","Nov 9, 2021",14.99,0,0,0,1,0,PDF,1,3,15
1,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",6.99,0,0,0,1,0,PDF,0,2,2
2,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",20.99,1,0,1,0,1,"Softcover, Premium Color Book (8.25"" x 10.75"")",0,2,2
3,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",14.99,1,0,0,0,1,"Softcover, Standard Color Book (8.25"" x 10.75"")",0,2,2
4,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",27.98,1,0,1,1,1,"PDF + Softcover, Premium Color Book (8.25"" x 1...",0,2,2


In [3]:
minPrice_df = df[["title", "price"]].groupby("title").min()
minPrice_df.head()

,price
title,
A Chill in Abashiri - A 1920s Taisho-Era Japan Scenario,4.95
Alone Against Nyarlathotep,18.99
Alone Against the Dark,6.95
Alone Against the Static,14.99
At the Gates of Carcosa,3.00


In [4]:
df2 = pandas.merge(df, minPrice_df, left_on="title", right_index=True).rename(columns={"price_y":"min price", "price_x":"price"})
df2.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,pdf,color,product,Greg Stafford,number of authors,number of artists,min price
0,RuneQuest Starter Set,https://www.drivethrurpg.com/en/product/370454...,180.0,11/10/21,"RuneQuest, BRP (Basic Roleplaying)","Nov 9, 2021",14.99,0,0,0,1,0,PDF,1,3,15,14.99
1,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",6.99,0,0,0,1,0,PDF,0,2,2,6.99
2,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",20.99,1,0,1,0,1,"Softcover, Premium Color Book (8.25"" x 10.75"")",0,2,2,6.99
3,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",14.99,1,0,0,0,1,"Softcover, Standard Color Book (8.25"" x 10.75"")",0,2,2,6.99
4,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",27.98,1,0,1,1,1,"PDF + Softcover, Premium Color Book (8.25"" x 1...",0,2,2,6.99


# simple linear model

In [5]:
import sklearn.linear_model
lm = sklearn.linear_model.LinearRegression().fit(df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']], df2['price'])
lm

LinearRegression()

In [6]:
lm.coef_

array([ 0.08406238, 17.19421425, 30.67786445,  9.05871987,  7.83046   ,
       -4.18474303])

In [7]:
import statsmodels.api
import numpy
df2['constant'] = 1
#lm = statsmodels.api.OLS(numpy.asarray(df2['price']), numpy.asarray(df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']])).fit()
#lm = statsmodels.api.OLS(df2['price'].to_list(), df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color']].to_dict("list")).fit()
x = df2[['page_count', 'softcover', 'hardcover', 'premium', 'pdf', 'color', 'constant']]
y = df2['price']
lm = statsmodels.api.OLS(y, x).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                     126.7
Date:                Mon, 13 May 2024   Prob (F-statistic):           1.68e-59
Time:                        15:41:54   Log-Likelihood:                -586.68
No. Observations:                 174   AIC:                             1187.
Df Residuals:                     167   BIC:                             1209.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0841      0.005     17.080      0.000       0.074       0.094
softcover     17.1942      2.597      6.620      0.000      12.066      22.322
hardcover     30.6779      2.765     11.096      0.000      25.220      36.136
premium        9.0587      1.613      5.616      0.000       5.874      12.244
pdf            7.8305      1.440      5.437      0.000       4.987      10.674
color         -4.1847      2.552     -1.640      0.103      -9.223       0.854
constant      -7.5031      1.737     -4.320      0.000     -10.932      -4.074
==============================================================================
Omnibus:                       24.102   Durbin-Watson:                   1.530
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.418
Skew:                           0.605   Prob(JB):                     2.51e-12
Kurtosis:                       5.430   Cond. No.                     1.32e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Why is the color coefficient negative?  Let's look at the color products...

In [8]:
df['color'].value_counts()

color
0    92
1    82
Name: count, dtype: int64

In [9]:
df.groupby(['color','hardcover']).count()['title']

color  hardcover
0      0            88
       1             4
1      0            60
       1            22
Name: title, dtype: int64

I would say that almost all physical books are in color, and it may be mixing some of the physical copy premium into the color coefficient.

In [10]:
df2['softcovern'] = df2['softcover'] * df2['page_count']
df2['hardcovern'] = df2['hardcover'] * df2['page_count']
df2['premiumn'] = df2['premium'] * df2['page_count']
df2['colorn'] = df2['color'] * df2['page_count']
df2['pdfn'] = df2['pdf'] * df2['page_count']
df2['softcoverb'] = df2['softcover'] * df2['min price']
df2['hardcoverb'] = df2['hardcover'] * df2['min price']
df2['premiumb'] = df2['premium'] * df2['min price']
df2['colorb'] = df2['color'] * df2['min price']
df2['pdfb'] = df2['pdf'] * df2['min price']
df2.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,softcovern,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb
0,RuneQuest Starter Set,https://www.drivethrurpg.com/en/product/370454...,180.0,11/10/21,"RuneQuest, BRP (Basic Roleplaying)","Nov 9, 2021",14.99,0,0,0,...,0.0,0.0,0.0,0.0,180.0,0.00,0.0,0.00,0.00,14.99
1,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",6.99,0,0,0,...,0.0,0.0,0.0,0.0,40.0,0.00,0.0,0.00,0.00,6.99
2,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",20.99,1,0,1,...,40.0,0.0,40.0,40.0,0.0,6.99,0.0,6.99,6.99,0.00
3,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",14.99,1,0,0,...,40.0,0.0,0.0,40.0,0.0,6.99,0.0,0.00,6.99,0.00
4,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",27.98,1,0,1,...,40.0,0.0,40.0,40.0,40.0,6.99,0.0,6.99,6.99,6.99


In [11]:
df2['priceExcess'] = df2['price'] - df2['min price']
df3 = df2.loc[df2['priceExcess'] != 0.0]
df3.head()

,title,url,page_count,published_date,system,modified_date,price,softcover,hardcover,premium,...,hardcovern,premiumn,colorn,pdfn,softcoverb,hardcoverb,premiumb,colorb,pdfb,priceExcess
2,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",20.99,1,0,1,...,0.0,40.0,40.0,0.0,6.99,0.0,6.99,6.99,0.00,14.00
3,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",14.99,1,0,0,...,0.0,0.0,40.0,0.0,6.99,0.0,0.00,6.99,0.00,8.00
4,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",27.98,1,0,1,...,0.0,40.0,40.0,40.0,6.99,0.0,6.99,6.99,6.99,20.99
5,Going Underground - Rivers of London RPG Scenario,https://www.drivethrurpg.com/en/product/475104...,40.0,4/19/24,Basic Roleplaying,"Apr 16, 2024",21.98,1,0,0,...,0.0,0.0,40.0,40.0,6.99,0.0,0.00,6.99,6.99,14.99
10,Heinrich's Call of Cthulhu Guide to Carcosa,https://www.drivethrurpg.com/en/product/462154...,340.0,11/29/23,Call of Cthulhu,"Jan 23, 2024",64.95,1,0,1,...,0.0,340.0,340.0,0.0,12.95,0.0,12.95,12.95,0.00,52.00


In [12]:
lm2 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'softcovern', 'hardcovern', 'premiumn', 'colorn', 'pdfn', 'constant']]).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     167.9
Date:                Mon, 13 May 2024   Prob (F-statistic):           8.21e-43
Time:                        15:41:54   Log-Likelihood:                -268.12
No. Observations:                  92   AIC:                             548.2
Df Residuals:                      86   BIC:                             563.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0294      0.005      6.046      0.000       0.020       0.039
softcovern    -0.0005      0.004     -0.128      0.898      -0.008       0.007
hardcovern     0.0299      0.004      7.533      0.000       0.022       0.038
premiumn       0.0870      0.007     12.486      0.000       0.073       0.101
colorn         0.0058      0.008      0.749      0.456      -0.010       0.021
pdfn           0.0565      0.006      9.423      0.000       0.045       0.068
constant       5.2791      0.724      7.290      0.000       3.840       6.719
==============================================================================
Omnibus:                       26.978   Durbin-Watson:                   1.437
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.107
Skew:                           1.044   Prob(JB):                     5.38e-14
Kurtosis:                       6.403   Cond. No.                     5.82e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.77e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

This suggests that color and softcover doesn't reflect an increase in price.  Lets try a model without either.

In [13]:
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'hardcovern', 'premiumn', 'pdfn', 'constant']]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     210.8
Date:                Mon, 13 May 2024   Prob (F-statistic):           6.99e-44
Time:                        15:41:54   Log-Likelihood:                -268.42
No. Observations:                  92   AIC:                             546.8
Df Residuals:                      87   BIC:                             559.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0322      0.007      4.942      0.000       0.019       0.045
hardcovern     0.0310      0.006      5.137      0.000       0.019       0.043
premiumn       0.0889      0.006     13.764      0.000       0.076       0.102
pdfn           0.0565      0.006      9.447      0.000       0.045       0.068
constant       5.3488      0.716      7.467      0.000       3.925       6.773
==============================================================================
Omnibus:                       23.254   Durbin-Watson:                   1.430
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.321
Skew:                           0.946   Prob(JB):                     8.74e-11
Kurtosis:                       5.917   Cond. No.                         313.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'premiumn', 'pdfn', 'constant', 'hardcover']]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     258.1
Date:                Mon, 13 May 2024   Prob (F-statistic):           2.26e-47
Time:                        15:41:54   Log-Likelihood:                -259.90
No. Observations:                  92   AIC:                             529.8
Df Residuals:                      87   BIC:                             542.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0378      0.005      7.068      0.000       0.027       0.048
premiumn       0.0864      0.006     14.617      0.000       0.075       0.098
pdfn           0.0565      0.005     10.363      0.000       0.046       0.067
constant       4.3156      0.654      6.602      0.000       3.016       5.615
hardcover      7.5088      1.068      7.032      0.000       5.386       9.631
==============================================================================
Omnibus:                       18.095   Durbin-Watson:                   1.419
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.863
Skew:                           0.807   Prob(JB):                     3.28e-07
Kurtosis:                       5.277   Cond. No.                         469.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Summary
Offering any extra starts with a cost of \\$4.32 + \\$0.04 per page.  This is probably for a softcover, which I suspect is why the softcover term isn't significant in this analysis.  Hardcover copy costs \\$7.5 beyond that cost, with \\$0.86 per page for premium printing.

In [15]:
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'premiumn', 'pdfn', 'hardcover', 'min price']]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            priceExcess   R-squared (uncentered):                   0.975
Model:                            OLS   Adj. R-squared (uncentered):              0.973
Method:                 Least Squares   F-statistic:                              672.3
Date:                Mon, 13 May 2024   Prob (F-statistic):                    6.58e-68
Time:                        15:41:54   Log-Likelihood:                         -250.46
No. Observations:                  92   AIC:                                      510.9
Df Residuals:                      87   BIC:                                      523.5
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0161      0.006      2.600      0.011       0.004       0.028
premiumn       0.0886      0.005     16.670      0.000       0.078       0.099
pdfn           0.0565      0.005     11.483      0.000       0.047       0.066
hardcover      5.8496      1.001      5.841      0.000       3.859       7.840
min price      0.7638      0.089      8.564      0.000       0.587       0.941
==============================================================================
Omnibus:                        2.865   Durbin-Watson:                   1.329
Prob(Omnibus):                  0.239   Jarque-Bera (JB):                2.265
Skew:                          -0.246   Prob(JB):                        0.322
Kurtosis:                       3.591   Cond. No.                         486.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# now consider models with feature as percentage of base price
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['page_count', 'premiumn', 'pdfn', 'hardcover', 'min price', "softcover", "hardcovern", "softcovern", "softcoverb", "hardcoverb", "premiumb", "premium", "color", "colorb", "colorn", "pdf", "pdfn", "pdfb", "constant"]]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            priceExcess   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     271.3
Date:                Mon, 13 May 2024   Prob (F-statistic):           2.05e-59
Time:                        15:41:54   Log-Likelihood:                -197.16
No. Observations:                  92   AIC:                             424.3
Df Residuals:                      77   BIC:                             462.1
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
page_count     0.0257      0.005      4.964      0.000       0.015       0.036
premiumn       0.1099      0.009     12.471      0.000       0.092       0.127
pdfn           0.0009      0.003      0.256      0.799      -0.006       0.008
hardcover     -1.4802      1.239     -1.195      0.236      -3.946       0.986
min price      0.2403      0.120      2.004      0.049       0.001       0.479
softcover      1.2735      0.933      1.366      0.176      -0.583       3.130
hardcovern     0.0026      0.004      0.610      0.543      -0.006       0.011
softcovern     0.0231      0.004      5.183      0.000       0.014       0.032
softcoverb    -0.3874      0.106     -3.657      0.000      -0.598      -0.176
hardcoverb     0.6277      0.100      6.256      0.000       0.428       0.828
premiumb      -0.6908      0.223     -3.099      0.003      -1.135      -0.247
premium        2.0360      1.310      1.554      0.124      -0.572       4.644
color         -0.4220      2.277     -0.185      0.853      -4.955       4.111
colorb         0.5332      0.250      2.130      0.036       0.035       1.032
colorn        -0.0060      0.010     -0.608      0.545      -0.025       0.014
pdf            0.2476      1.026      0.241      0.810      -1.795       2.291
pdfn           0.0009      0.003      0.256      0.799      -0.006       0.008
pdfb           0.9269      0.150      6.185      0.000       0.628       1.225
constant      -0.2067      1.484     -0.139      0.890      -3.162       2.749
==============================================================================
Omnibus:                       14.156   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               39.008
Skew:                           0.366   Prob(JB):                     3.39e-09
Kurtosis:                       6.105   Cond. No.                     3.01e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.49e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

hardcover, softcover, premium, color, and pdf are all not significant.
The % of base cost coefficients are all significant, though the softcover % is negative.
lets take out the not significant coefficients

In [32]:
# now consider models with feature as percentage of base price
lm3 = statsmodels.api.OLS(df3['priceExcess'], df3[['softcoverb', "hardcoverb", "premiumb", "pdfb", "colorn"]]).fit()
lm3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            priceExcess   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.955
Method:                 Least Squares   F-statistic:                              392.8
Date:                Mon, 13 May 2024   Prob (F-statistic):                    4.19e-58
Time:                        15:49:06   Log-Likelihood:                         -274.36
No. Observations:                  92   AIC:                                      558.7
Df Residuals:                      87   BIC:                                      571.3
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
softcoverb     0.3897      0.103      3.793      0.000       0.186       0.594
hardcoverb     0.8280      0.111      7.477      0.000       0.608       1.048
premiumb       0.8145      0.105      7.742      0.000       0.605       1.024
pdfb           0.9718      0.099      9.858      0.000       0.776       1.168
colorn         0.0584      0.006     10.063      0.000       0.047       0.070
==============================================================================
Omnibus:                       39.377   Durbin-Watson:                   0.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.794
Skew:                           1.592   Prob(JB):                     3.17e-20
Kurtosis:                       6.644   Cond. No.                         41.4
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Summary
This model has a 96% R^2 and is both simple and intuitive.  Softcover B&W are priced at a premium over just PDF of 39+-10\% of the base price.  Hardcovers are priced at 83+-11\% of the base price.  Premium printing costs an additional 81+-10\% of the base price above that.  Color printing is priced at just \\$0.06 per page, and offering a PDF in addition to print is priced at 97+-10\% of the base price (i.e. almost exactly the PDF price without discount).